## 8. 転移学習とファインチューニング
学習済みの良いパラメータを利用したニューラルネットワークの学習に転移学習やファインチューニングがある。  
出力層のみ学習させる手法を**転移学習**、出力層以外の畳み込み層なども学習させる手法を**ファインチューニング**と呼ぶ。

#### keras
#### 転移学習

In [208]:
import tensorflow.keras as keras
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D,Input, Flatten, Dropout
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from keras.applications.vgg16 import VGG16
#from keras.applications.resnet_v2 import ResNet101V2
#from keras.applications.inception_v3 import InceptionV3
#from keras.applications.inception_resnet_v2 import InceptionResNetV2
#from keras.applications.xception import Xception
#from keras.applications.nasnet import NASNetLarge
#from keras.applications.densenet import DenseNet121
#from keras.applications.densenet import DenseNet201
#from keras.applications.mobilenet_v2 import MobileNetV2

In [212]:
train_dir = "C:/Users/tanak/study/参考書/pytorchによる発展ディープラーニング/pytorch_advanced-master/1_image_classification/data/hymenoptera_data/train/"
val_dir = "C:/Users/tanak/study/参考書/pytorchによる発展ディープラーニング/pytorch_advanced-master/1_image_classification/data/hymenoptera_data/val/"

In [225]:
classes = os.listdir(train_dir)
image_size = 224

def preprocess(dir):
    x = []
    y = []

    for idx, cls in enumerate(classes):
        print(cls)
        file_lists = glob.glob(os.path.join(dir, classes[idx]) + "/*.jpg")
        for file in file_lists:
            image = Image.open(file)
            image = image.convert("RGB")    
            image = image.resize((image_size, image_size))
            data = np.asarray(image)
            x.append(data)
            y.append(idx)

    x = np.array(x)
    y = np.array(y)

    x = x.astype('float32')
    x /= 255

    y = keras.utils.to_categorical(y, num_classes)
    return x, y

x_train, y_train = preprocess(train_dir)
x_val, y_val = preprocess(val_dir)

ants
bees
ants
bees


In [280]:
num_classes  = 2
size = 224
batch_size = 32

input_tensor = Input(shape=(size, size, 3))
base_model = VGG16(weights='imagenet', include_top=False,input_tensor=input_tensor)

base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [281]:
last = base_model.output
x = Flatten()(last)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)

prediction = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=prediction)

In [282]:
for layer in base_model.layers:
    layer.trainable = False

In [283]:
optimizer = SGD(learning_rate=0.001, momentum=0.9, decay=5e-4)
loss = "binary_crossentropy"

In [284]:
model.compile(loss=loss,
              optimizer=optimizer,
              metrics=['accuracy'])

In [285]:
patience = 3

earlystop = EarlyStopping(patience=patience)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', # tensorflow 2.x
                                            verbose=1, 
                                            factor=1/num_classes, # 1/ class数
                                            min_lr=0.00001)

modelCheckpoint = ModelCheckpoint(filepath = ".",
                                  monitor='val_accuracy',
                                  verbose=1,
                                  save_best_only=True,
                                  save_weights_only=False, # 重みのみ保存
                                  mode='max', # val_accuracyの場合
                                  save_freq=1)

callbacks = [earlystop]

In [286]:
epochs = 15

history = model.fit(
        x_train, y_train,
        batch_size=batch_size,
        initial_epoch = 0,
        epochs=epochs,
        validation_data=(x_val, y_val),
        shuffle=True,
        callbacks=callbacks)

Epoch 1/15
8/8 [==============================] - 95s 12s/step - loss: 1.6796 - accuracy: 0.4650 - val_loss: 1.4208 - val_accuracy: 0.5425
Epoch 2/15
8/8 [==============================] - 97s 12s/step - loss: 1.2767 - accuracy: 0.5597 - val_loss: 0.4806 - val_accuracy: 0.8039
Epoch 3/15
8/8 [==============================] - 97s 12s/step - loss: 0.5544 - accuracy: 0.7613 - val_loss: 0.9571 - val_accuracy: 0.6209
Epoch 4/15
8/8 [==============================] - 98s 12s/step - loss: 0.5014 - accuracy: 0.7942 - val_loss: 0.5221 - val_accuracy: 0.7516
Epoch 5/15
8/8 [==============================] - 96s 12s/step - loss: 0.3563 - accuracy: 0.8560 - val_loss: 0.4264 - val_accuracy: 0.8301
Epoch 6/15
8/8 [==============================] - 96s 12s/step - loss: 0.2925 - accuracy: 0.8848 - val_loss: 0.7429 - val_accuracy: 0.6797
Epoch 7/15
8/8 [==============================] - 98s 12s/step - loss: 0.2573 - accuracy: 0.8765 - val_loss: 0.3781 - val_accuracy: 0.8562
Epoch 8/15
8/8 [===========

In [289]:
model.save('keras_teni_model')

INFO:tensorflow:Assets written to: keras_teni_model\assets


#### ファインチューニング

In [290]:
num_classes  = 2
size = 224
batch_size = 32

input_tensor = Input(shape=(size, size, 3))
base_model = VGG16(weights='imagenet', include_top=False,input_tensor=input_tensor)

base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [291]:
last = base_model.output
x = Flatten()(last)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)

prediction = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=prediction)

In [292]:
for layer in base_model.layers[:15]:
    layer.trainable = False

for layer in base_model.layers[15:]:
    layer.trainable = True

In [293]:
optimizer = SGD(learning_rate=0.001, momentum=0.9, decay=5e-4)
loss = "binary_crossentropy"

In [294]:
model.compile(loss=loss,
              optimizer=optimizer,
              metrics=['accuracy'])

In [295]:
patience = 3

earlystop = EarlyStopping(patience=patience)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', # tensorflow 2.x
                                            verbose=1, 
                                            factor=1/num_classes, # 1/ class数
                                            min_lr=0.00001)

modelCheckpoint = ModelCheckpoint(filepath = ".",
                                  monitor='val_accuracy',
                                  verbose=1,
                                  save_best_only=True,
                                  save_weights_only=False, # 重みのみ保存
                                  mode='max', # val_accuracyの場合
                                  save_freq=1)

callbacks = [earlystop]

In [296]:
epochs = 15

history = model.fit(
        x_train, y_train,
        batch_size=batch_size,
        initial_epoch = 0,
        epochs=epochs,
        validation_data=(x_val, y_val),
        shuffle=True,
        callbacks=callbacks)

Epoch 1/15
8/8 [==============================] - 81s 10s/step - loss: 0.8590 - accuracy: 0.5391 - val_loss: 0.5313 - val_accuracy: 0.7320
Epoch 2/15
8/8 [==============================] - 109s 14s/step - loss: 0.4818 - accuracy: 0.7572 - val_loss: 0.4873 - val_accuracy: 0.7712
Epoch 3/15
8/8 [==============================] - 114s 14s/step - loss: 0.2633 - accuracy: 0.8889 - val_loss: 0.4003 - val_accuracy: 0.8431
Epoch 4/15
8/8 [==============================] - 108s 13s/step - loss: 0.1840 - accuracy: 0.9095 - val_loss: 0.3781 - val_accuracy: 0.8693
Epoch 5/15
8/8 [==============================] - 109s 14s/step - loss: 0.1282 - accuracy: 0.9506 - val_loss: 0.4282 - val_accuracy: 0.8497
Epoch 6/15
8/8 [==============================] - 108s 13s/step - loss: 0.0913 - accuracy: 0.9671 - val_loss: 0.4753 - val_accuracy: 0.8627
Epoch 7/15
8/8 [==============================] - 107s 13s/step - loss: 0.0517 - accuracy: 0.9877 - val_loss: 0.4873 - val_accuracy: 0.8758


In [297]:
model.save('keras_finetuning_model')

INFO:tensorflow:Assets written to: keras_finetuning_model\assets


#### ptorch

In [134]:
import os
import glob
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers
import torch.utils.data as data
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [78]:
"""
root = os.path.join('-', '.torch', 'mnist')
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize((128, 128))])
mnist_train = datasets.CIFAR10(root=root,
                             download=True,
                             train=True, # 訓練データの取得
                             transform=transform)
mnist_test = datasets.CIFAR10(root=root,
                            download=True,
                            train=False,
                            transform=transform)
    
n_samples = len(mnist_train)
n_train = int(n_samples * 0.8)
n_val = n_samples - n_train
    
mnist_train, mnist_val = random_split(mnist_train, [n_train, n_val])
    
train_dataloader = DataLoader(mnist_train,
                              batch_size=100,
                              shuffle=True) # 各エポックでデータのシャッフルを行う
val_dataloader = DataLoader(mnist_val,
                            batch_size=100,
                            shuffle=False)
test_dataloader = DataLoader(mnist_test,
                             batch_size=100,
                             shuffle=False)
"""

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting -\.torch\mnist\cifar-10-python.tar.gz to -\.torch\mnist
Files already downloaded and verified


In [168]:
%cd C:\Users\tanak\study\参考書\pytorchによる発展ディープラーニング\pytorch_advanced-master\1_image_classification

class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(
                    resize ,scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }
    
    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)
    
def make_datapath_list(phase='train'):
    root_path = "./data/hymenoptera_data/"
    target_path = os.path.join(root_path+phase+'/**/*.jpg')
    print(target_path)
    
    path_list = []
    for path in glob.glob(target_path):
        path_list.append(path)
    
    return path_list

train_list = make_datapath_list(phase='train')
val_list = make_datapath_list(phase='val')


class HymenopteraDataset(data.Dataset):
    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        
        img_transformed = self.transform(img, self.phase)
        
        if self.phase == 'train':
            label = img_path[30:34]
        elif self.phase == 'val':
            label = img_path[30:34]
        
        if label == 'ants':
            label = 0
        elif label == 'bees':
            label = 1

        return img_transformed, label

size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_dataset = HymenopteraDataset(file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')
val_dataset = HymenopteraDataset(file_list=train_list, transform=ImageTransform(size, mean, std), phase='val')

C:\Users\tanak\study\参考書\pytorchによる発展ディープラーニング\pytorch_advanced-master\1_image_classification
./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


In [169]:
batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [170]:
batch_iterator = iter(val_dataloader)
inputs, labels = next(batch_iterator)
print(inputs.size())
print(labels)

torch.Size([32, 3, 224, 224])
tensor([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
        0, 1, 1, 1, 1, 0, 0, 0])


#### 転移学習

In [171]:
# モデルの読み込み
#net = models.resnet18(pretrained=True)
#net = models.alexnet(pretrained=True)
net = models.vgg16(pretrained=True)
#net = models.squeezenet1_0(pretrained=True)
#net = models.densenet161(pretrained=True)
#net = models.inception_v3(pretrained=True)
#net = models.googlenet(pretrained=True)
#net = models.shufflenet_v2_x1_0(pretrained=True)
#net = models.mobilenet_v2(pretrained=True)
#net = models.mobilenet_v3_large(pretrained=True)
#net = models.mobilenet_v3_small(pretrained=True)
#net = models.resnext50_32x4d(pretrained=True)
#net = models.wide_resnet50_2(pretrained=True)
#net = models.mnasnet1_0(pretrained=True)

In [172]:
# 勾配を計算させない
for param in net.parameters():
    param.requires_grad = False
net.to(device)

# 全結合層の変更・この層のみを学習させる
num_frts = net.classifier[6].in_features
print(net.classifier[6])
net.classifier[6] = nn.Linear(in_features=num_frts, out_features=2)
print(net.classifier[6])

Linear(in_features=4096, out_features=1000, bias=True)
Linear(in_features=4096, out_features=2, bias=True)


In [173]:
criterion = nn.CrossEntropyLoss()
optimizer = optimizers.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [175]:
class EarlyStopping:
    '''
    早期終了 (early stopping)
    定義する必要がある
    '''
    def __init__(self, patience=0, verbose=0):
        self._step = 0
        self._loss = float('inf')
        self.patience = patience
        self.verbose = verbose

    def __call__(self, loss):
        if self._loss < loss:
            self._step += 1
            if self._step > self.patience:
                if self.verbose:
                    print('early stopping')
                return True
        else:
            self._step = 0
            self._loss = loss

        return False

def compute_loss(t, y):
    return criterion(y, t)
    
def train_step(x, t):
    net.train()
    preds = net(x)
    loss = compute_loss(t, preds)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss, preds
    
def val_step(x, t):
    net.eval()
    preds = net(x)
    loss = criterion(preds, t)
    return loss, preds
    
epochs = 2
hist = {'loss': [], 'accuracy': [],
        'val_loss': [], 'val_accuracy': []}
es = EarlyStopping(patience=2, verbose=1)
    
for epoch in range(epochs):
    train_loss = 0.
    train_acc = 0.
    val_loss = 0.
    val_acc = 0.
        
    for (x, t) in train_dataloader:
        x, t = x.to(device), t.to(device)
        loss, preds = train_step(x, t)
        train_loss += loss.item()
        train_acc += accuracy_score(t.tolist(), preds.argmax(dim=-1).tolist())
        
    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataloader)
        
    for (x, t) in val_dataloader:
        x, t = x.to(device), t.to(device)
        loss, preds = train_step(x, t)
        val_loss += loss.item()
        val_acc += accuracy_score(t.tolist(), preds.argmax(dim=-1).tolist())
            
    val_loss /= len(val_dataloader)
    val_acc /= len(val_dataloader)
        
    hist['loss'].append(train_loss)
    hist['accuracy'].append(train_acc)
    hist['val_loss'].append(val_loss)
    hist['val_accuracy'].append(val_acc)
        
    print('epoch: {}, loss: {:.3f}, acc: {:.3f}, val_loss: {:.3f}, val_acc: {:.3f}'.format(epoch+1, train_loss, train_acc, val_loss, val_acc))
        
    if es(val_loss): # 早期終了判定
        break

epoch: 1, loss: 0.352, acc: 0.888, val_loss: 0.249, val_acc: 0.943
epoch: 2, loss: 0.089, acc: 0.977, val_loss: 0.130, val_acc: 0.965


In [177]:
torch.save(net.to('cpu').state_dict(), 'cnn_teni_model_pytorch.pth')

#### ファインチューニング

In [178]:
# モデルの読み込み
#net = models.resnet18(pretrained=True)
#net = models.alexnet(pretrained=True)
net = models.vgg16(pretrained=True)
#net = models.squeezenet1_0(pretrained=True)
#net = models.densenet161(pretrained=True)
#net = models.inception_v3(pretrained=True)
#net = models.googlenet(pretrained=True)
#net = models.shufflenet_v2_x1_0(pretrained=True)
#net = models.mobilenet_v2(pretrained=True)
#net = models.mobilenet_v3_large(pretrained=True)
#net = models.mobilenet_v3_small(pretrained=True)
#net = models.resnext50_32x4d(pretrained=True)
#net = models.wide_resnet50_2(pretrained=True)
#net = models.mnasnet1_0(pretrained=True)

In [179]:
net = net.to(device)

criterion = nn.CrossEntropyLoss()

In [180]:
from torchsummary import summary

print(summary(net, (3, 256, 256)))
print(net)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
              ReLU-2         [-1, 64, 256, 256]               0
            Conv2d-3         [-1, 64, 256, 256]          36,928
              ReLU-4         [-1, 64, 256, 256]               0
         MaxPool2d-5         [-1, 64, 128, 128]               0
            Conv2d-6        [-1, 128, 128, 128]          73,856
              ReLU-7        [-1, 128, 128, 128]               0
            Conv2d-8        [-1, 128, 128, 128]         147,584
              ReLU-9        [-1, 128, 128, 128]               0
        MaxPool2d-10          [-1, 128, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]         295,168
             ReLU-12          [-1, 256, 64, 64]               0
           Conv2d-13          [-1, 256, 64, 64]         590,080
             ReLU-14          [-1, 256,

In [181]:
# 全結合層の変更
print(net.classifier[6])
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)
print(net.classifier[6])

Linear(in_features=4096, out_features=1000, bias=True)
Linear(in_features=4096, out_features=2, bias=True)


In [182]:
for name, param in net.named_parameters():
    print('name : ', name)

name :  features.0.weight
name :  features.0.bias
name :  features.2.weight
name :  features.2.bias
name :  features.5.weight
name :  features.5.bias
name :  features.7.weight
name :  features.7.bias
name :  features.10.weight
name :  features.10.bias
name :  features.12.weight
name :  features.12.bias
name :  features.14.weight
name :  features.14.bias
name :  features.17.weight
name :  features.17.bias
name :  features.19.weight
name :  features.19.bias
name :  features.21.weight
name :  features.21.bias
name :  features.24.weight
name :  features.24.bias
name :  features.26.weight
name :  features.26.bias
name :  features.28.weight
name :  features.28.bias
name :  classifier.0.weight
name :  classifier.0.bias
name :  classifier.3.weight
name :  classifier.3.bias
name :  classifier.6.weight
name :  classifier.6.bias


In [183]:
# featureモジュール
params_to_update_1 = []
# classifierモジュール(後半)
params_to_update_2 = []
# classifierモジュール(付け替えた層)
params_to_update_3 = []

# 学習させる層のパラメータ名を指定
update_param_names_1 = ['features']
update_param_names_2 = ['classifier.0.weight', 'classifier.0.bias',
                        'classifier.3.weight', 'classifier.3.bias']
update_param_names_3 = ['classifier.6.weight', 'classifier.6.bias']

# パラメータごとに各リストに格納
for name, param in net.named_parameters():

    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1に格納：", name)
    
    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2に格納：", name)
    
    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3に格納：", name)

params_to_update_1に格納： features.0.weight
params_to_update_1に格納： features.0.bias
params_to_update_1に格納： features.2.weight
params_to_update_1に格納： features.2.bias
params_to_update_1に格納： features.5.weight
params_to_update_1に格納： features.5.bias
params_to_update_1に格納： features.7.weight
params_to_update_1に格納： features.7.bias
params_to_update_1に格納： features.10.weight
params_to_update_1に格納： features.10.bias
params_to_update_1に格納： features.12.weight
params_to_update_1に格納： features.12.bias
params_to_update_1に格納： features.14.weight
params_to_update_1に格納： features.14.bias
params_to_update_1に格納： features.17.weight
params_to_update_1に格納： features.17.bias
params_to_update_1に格納： features.19.weight
params_to_update_1に格納： features.19.bias
params_to_update_1に格納： features.21.weight
params_to_update_1に格納： features.21.bias
params_to_update_1に格納： features.24.weight
params_to_update_1に格納： features.24.bias
params_to_update_1に格納： features.26.weight
params_to_update_1に格納： features.26.bias
params_to_update_1に格納： f

In [184]:
optimizer = optimizers.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3},
], momentum=0.9)

In [185]:
class EarlyStopping:
    '''
    早期終了 (early stopping)
    定義する必要がある
    '''
    def __init__(self, patience=0, verbose=0):
        self._step = 0
        self._loss = float('inf')
        self.patience = patience
        self.verbose = verbose

    def __call__(self, loss):
        if self._loss < loss:
            self._step += 1
            if self._step > self.patience:
                if self.verbose:
                    print('early stopping')
                return True
        else:
            self._step = 0
            self._loss = loss

        return False

def compute_loss(t, y):
    return criterion(y, t)
    
def train_step(x, t):
    net.train()
    preds = net(x)
    loss = compute_loss(t, preds)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss, preds
    
def val_step(x, t):
    net.eval()
    preds = net(x)
    loss = criterion(preds, t)
    return loss, preds
    
epochs = 2
hist = {'loss': [], 'accuracy': [],
        'val_loss': [], 'val_accuracy': []}
es = EarlyStopping(patience=2, verbose=1)
    
for epoch in range(epochs):
    train_loss = 0.
    train_acc = 0.
    val_loss = 0.
    val_acc = 0.
        
    for (x, t) in train_dataloader:
        x, t = x.to(device), t.to(device)
        loss, preds = train_step(x, t)
        train_loss += loss.item()
        train_acc += accuracy_score(t.tolist(), preds.argmax(dim=-1).tolist())
        
    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataloader)
        
    for (x, t) in val_dataloader:
        x, t = x.to(device), t.to(device)
        loss, preds = train_step(x, t)
        val_loss += loss.item()
        val_acc += accuracy_score(t.tolist(), preds.argmax(dim=-1).tolist())
            
    val_loss /= len(val_dataloader)
    val_acc /= len(val_dataloader)
        
    hist['loss'].append(train_loss)
    hist['accuracy'].append(train_acc)
    hist['val_loss'].append(val_loss)
    hist['val_accuracy'].append(val_acc)
        
    print('epoch: {}, loss: {:.3f}, acc: {:.3f}, val_loss: {:.3f}, val_acc: {:.3f}'.format(epoch+1, train_loss, train_acc, val_loss, val_acc))
        
    if es(val_loss): # 早期終了判定
        break

epoch: 1, loss: 0.494, acc: 0.771, val_loss: 0.151, val_acc: 0.939
epoch: 2, loss: 0.067, acc: 0.977, val_loss: 0.069, val_acc: 0.979


In [186]:
torch.save(net.to('cpu').state_dict(), 'cnn_finetuning_model_pytorch.pth')